In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Data for the table
data = {
    'Plant': ['Plant A', 'Plant B', 'Plant C', 'Plant D', 'Plant E'],
    'DC1': [180, 110, 145, 180, 135],
    'DC2': [215, 175, 165, 220, 165],
    'DC3': [65, 95, 150, 175, 160],
    'DC4': [55, 145, 95, 365, 65],
    'DC5': [105, 165, 150, 190, 130],
    'DC6': [110, 120, 180, 185, 150],
    'DC7': [100, 80, 195, 155, 10000],
    'DC8': [125, 160, 120, 265, 10000],
    'DC9': [155, 205, 135, 290, 10000]
}

# Create a DataFrame
costs = pd.DataFrame(data)
costs.set_index('Plant', inplace=True)

# Capacities at plants
capacities = {
    'Plant A': 1500,
    'Plant B': 800,
    'Plant C': 1190 * 0.5,
    'Plant D': 2900 * 0.5,  # Modify this row
    'Plant E': 1250  # Modify this row
}

# Requirements at DCs
requirements = {
    'DC1': 520,
    'DC2': 800,
    'DC3': 300,
    'DC4': 1450,
    'DC5': 375,
    'DC6': 100,
    'DC7': 935,
    'DC8': 750,
    'DC9': 620
}


no_plant_e = ['DC7', 'DC8', 'DC9']

In [3]:
m = gp.Model()
x = m.addVars(capacities.keys(), requirements.keys(), lb=0, name="Proportion")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2026-01-17


In [4]:
capacity = m.addConstrs((gp.quicksum(x[i, j] for j in requirements.keys()) <= capacities[i] for i in capacities.keys()), name="Capacity")

no_shipment = m.addConstrs((x['Plant E', dc] == 0 for dc in no_plant_e), name="No Shipment Plant E to DC 7, 8, 9")
m.update()

In [5]:
m.setObjective(gp.quicksum((requirements[j] - x[i, j]) for i in capacities for j in requirements), GRB.MINIMIZE)
m.update()

In [6]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 45 columns and 48 nonzeros
Model fingerprint: 0x519f46b0
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 2e+03]
Presolve removed 8 rows and 45 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3655000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.365500000e+04


In [8]:
cost = gp.quicksum(costs.loc[i, j] * x[i, j].X for i in capacities for j in requirements)

In [10]:
cost.getValue()

780575.0

### Option 2: Tighten Capacity Constraint, relax demand constraint

In [11]:
m2 = gp.Model()
x = m2.addVars(capacities.keys(), requirements.keys(), lb=0, name="Transit")
m2.update()

In [12]:
demand2 = m2.addConstrs((gp.quicksum(x[i, j] for i in capacities.keys()) <= requirements[j] for j in requirements.keys()), name="Demand")
capacity2 = m2.addConstrs((gp.quicksum(x[i, j] for j in requirements.keys()) == capacities[i] for i in capacities.keys()), name="Capacity")
no_shipment2 = m2.addConstrs((x['Plant E', dc] == 0 for dc in no_plant_e), name="No Shipment Plant E to DC 7, 8, 9")

m2.update()

In [13]:
m2.setObjective(gp.quicksum(costs.loc[i, j] * x[i, j] for i in capacities.keys() for j in requirements.keys()), GRB.MINIMIZE)
m2.update()

In [14]:
m2.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 17 rows, 45 columns and 93 nonzeros
Model fingerprint: 0x9e6d8332
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 14 rows, 42 columns, 84 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0902500e+05   4.715766e+02   0.000000e+00      0s
      11    6.5762500e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.576250000e+05


In [16]:
for v in m2.getVars():
    if v.X > 0:
        print(v.VarName, v.X)


print("Objective Value (Total Cost): ")
print(m2.ObjVal)

Transit[Plant A,DC3] 300.0
Transit[Plant A,DC4] 200.0
Transit[Plant A,DC5] 375.0
Transit[Plant A,DC8] 625.0
Transit[Plant B,DC7] 800.0
Transit[Plant C,DC8] 125.0
Transit[Plant C,DC9] 470.0
Transit[Plant D,DC1] 520.0
Transit[Plant D,DC2] 695.0
Transit[Plant D,DC6] 100.0
Transit[Plant D,DC7] 135.0
Transit[Plant E,DC4] 1250.0
Objective Value (Total Cost): 
657625.0


### Option 3: Multi-objective optimization with Pareto Optimality